# 模型实现的基本框架

* 定义获取训练集里类别先验概率的函数
* 将核心训练步骤以外的训练步骤进行定义，其中核心训练步骤需要训练出一个决策函数，该决策函数能够输出给定数据的后验概率。
* 利用决策函数定义预测函数和评估函数

In [4]:
import numpy as np

# 定义NaiveBayes的基类，方便后续拓展
class NaiveBayes:
    # 初始化
    def _init_(self):
        self._x = self._y = None       # 记录训练集的变量
        self._data = self._func = None # 核心数组（存储实际使用的条件概率信息）；模型的决策函数（根据输入x，y输出对应的后验概率）
        self._n_possibilities = None   # 记录各个纬度特征取值个数的数组
        self._labelled_x = self._label_zip = None # 记录按类别分开后的输入数据的数组；记录类别相关信息的数组
        self._cat_counter = self._con_counter = None # 记录第i类数据的个数；记录数据条件概率的原始极大似然估计
        self.label_dic = self._feat_dics = None # 记录数据数值化类别时的转换关系；用于记录数值化各纬度特征时的转换关系
    
    # 重载 _getitem_(self, item)
    def _getitem_(self, item):
        if isinstance(item, str):
            return getattr(self, "_" + item)
        
    # 留下抽象方法让子类定义，这里tar_idx参数和self._tar_idx意义一致
    def feed_data(self, x, y, tar_idx=None):
        pass
    
    # 留下抽象方法让子类定义，这里的sample_weight代表样本权重
    def feed_sample_weight(self, sample, sample_weight=None):
        pass
    
    # 定义计算先验概率的函数，lb就是各个估计中的平滑项
    # lb默认值为1，即默认采用拉普拉斯平滑
    def get_prior_probability(self, lb=1):
        return [(_c_num + lb) / (len(self._y) + lb * len(self._cat_counter)) for _c_num in self._cat_counter]
    
    # 定义具有普适性的训练函数
    